In [29]:
import os
import requests
from dotenv import load_dotenv
import time

In [18]:
load_dotenv()

url = "https://ai.nutanix.com/api/v1/chat/completions" 

nai_api_key = os.getenv("NAI_API_KEY")

# print(nai_api_key)

In [31]:
question = "produce the answer. We start with 15 eggs and if 5 are taken out how many are left? think step by step "


/Users/hritik.raj/DSPy/nai-api/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ai.nutanix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Time taken: 2.651865005493164 seconds
Response from the API:
{'id': 'd2d3943d-177f-4460-8286-6eaa80970c4f', 'object': 'chat.completion', 'created': 1729682716, 'model': 'vllm-llama-3-1', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "To determine how many eggs are left after 5 are taken out, we'll follow the steps below:\n\n1. We start with 15 eggs.\n2. Next, 5 eggs are taken out. To find out how many eggs are left, we subtract 5 from 15.\n3. 15 - 5 = 10\n\nSo, there are 10 eggs left."}, 'finish_reason': 'stop', 'content_filter_results': {'hate': {'filtered': False}, 'self_harm': {'filtered': False}, 'sexual': {'filtered': False}, 'violence': {'filtered': False}, 'jailbreak': {'filtered': False, 'detected': False}, 'profanity': {'filtered': False, 'detected': False}}}], 'usage': {'prompt_tokens': 62, 'completion_tokens': 79, 'total_tokens': 141, 'prompt_tokens_details': None, 'completion_tokens_details': None}, 'system_fingerprint': ''}


In [28]:
print(response.json()["choices"][0]["message"]["content"])

To find the number of eggs left, we start with 15 and subtract the 5 eggs that were taken out.

15 (initial eggs) - 5 (eggs taken out) = 10

There are 10 eggs left.


In [46]:
def get_response_llm(prompt):
    response = requests.post(url, 
                         headers={
                                "Authorization": f"Bearer {nai_api_key}", 
                                "accept": "application/json",
                                "Content-Type": "application/json"
                            }, 
                         json={
                                "model": "vllm-llama-3-1",
                                "messages": [
                                    {
                                        "role": "user",
                                        "content": f"{prompt}"
                                    }
                                ],
                                "max_tokens": 256,
                                "stream": False
                            }, 
                         verify=False)
    return response

# Step 1 : Refinning the prompt

### My primary refinning techniques would be:

- Clarity
- Context
- Conciseness

### Base Prompt?

Deciding on a base prompt is an important step, and it depends on the task we are actually trying to accomplish. Lets start with a data analysis example, assuming the client wants to generate insights from a sales report.

In [39]:
with open('prompts/prompt_01.prompt', 'r') as file:
    base_prompt = file.read()

print(base_prompt)

Analyze the following sales report and provide a summary of the top 3 performing products along with their respective revenue


### Lets Define Rules for clarity, context and conciseness

In [35]:
generic_rules={
    "clarity":{
        "summarization":"Summarize the following text cleary and coincisely.",
        "generation":"Generate a response that clearly addresses the request.",
        "QA":"Answer the Question based on the provided information."
    },
    "context":{
        "summarization":"The text is about [subject/Topic]",
        "generation":"Make sure to include context about [subject/topic]",
        "QA":"Use the context of [subject/topic] to provide a relevant answer."
    },
    "conciseness":{
        "summarization": "Limit the summary to under [word_count] words.",
        "generation": "Keep the response concise and focused on the main idea.",
        "QA": "Provide a direct and concise answer."
    }
}

Since we have defined the rules, maybe now we will have to classify the `prompt` among these

In [42]:
def classify_prompt(prompt):
    """Classify the prompt type based on the keywords or structure"""
    if "summarize" in prompt.lower():
        return "summarization"
    elif "generate" in prompt.lower():
        return "generation"
    elif "answer" in prompt.lower():
        return "QA"
    else:
        return "generic"

In [44]:
import re

def apply_clarity(prompt, subject=None):
    """Add clarity to any prompt, dynamically incorporating subject if provided"""
    clarity_template = "Provide a clear and concise response for the following task."
    if subject:
        clarity_template = f"Provide a clear and concise response related to {subject}."
    refined = clarity_template + " " + prompt
    return refined

def apply_context(prompt, context_info=None):
    """Dynamically apply context if available or use a general placeholder"""
    if context_info:
        context_template = f"Use the context of {context_info} to perform the task."
    else:
        context_template = "Use the given context to perform the task."
    return context_template + " " + prompt

def apply_conciseness(prompt, word_limit=100):
    """Ensure conciseness by dynamically applying word limits"""
    conciseness_template = f"Keep the response concise and under {word_limit} words."
    return conciseness_template + " " + prompt


In [45]:
def automate_refinement_generic(base_prompt, subject=None, context_info=None, word_limit=100):
    """Automate prompt refinement for any prompt type with dynamic subject, context, and conciseness rules"""
    
    refined_prompt = apply_clarity(base_prompt, subject)
    refined_prompt = apply_context(refined_prompt, context_info)
    refined_prompt = apply_conciseness(refined_prompt, word_limit)
    
    return refined_prompt

refined_prompt_generic = automate_refinement_generic(base_prompt)

refined_prompt_with_subject = automate_refinement_generic(base_prompt, subject="customer feedback", context_info="customer reviews", word_limit=150)

print("Refined Prompt (Generic):", refined_prompt_generic)
print("\nRefined Prompt (With Subject):", refined_prompt_with_subject)


Refined Prompt (Generic): Keep the response concise and under 100 words. Use the given context to perform the task. Provide a clear and concise response for the following task. Analyze the following data and provide insights.

Refined Prompt (With Subject): Keep the response concise and under 150 words. Use the context of customer reviews to perform the task. Provide a clear and concise response related to customer feedback. Analyze the following data and provide insights.


In [50]:
print(get_response_llm(refined_prompt_with_subject).json()["choices"][0]["message"]["content"])

/Users/hritik.raj/DSPy/nai-api/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ai.nutanix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


However, I don't see any data provided in your question. If you provide the customer review data, I'd be happy to analyze and provide insights. Please paste the data, and I'll get started.

If you have any example customer review ratings or feedback, you can provide it, and I'll give you a concise analysis under 150 words.

Additionally, what type of insights would you like to gain from the analysis? Are you looking for customer pain points, common issues, or areas of improvement?


In [51]:
from dspy import Dyanamic


ImportError: cannot import name 'DynamicPrompt' from 'dspy' (/Users/hritik.raj/DSPy/nai-api/lib/python3.12/site-packages/dspy/__init__.py)